# Exercise 5 <br>
Student: Duncan Ferguson <br>
Student Id: 871641260 <br>
Class: Comp 4431-1 <br>
Assignment: Exercise 5 <br>
Date: 10/16/2021 <br>
Group: Name: Broken Toe <br>
Group Members: Emma Bright, Mike Santoro <br>

In [57]:
import pandas as pd
from csv import reader
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

theData = [
['milk','onion','egg','bread','chili'],
['milk','egg','bread','chili'],
['milk','garlic','beans','chili'],
['milk','egg','beans','chili'],
['milk','onion','egg','bread'],
['milk','onion','egg'],
['garlic','onion','beans'],
['garlic','onion','beans'],
['garlic','onion','beans'],
['garlic','onion','beans'],
['chili','garlic','onion','beans'],
['chili','garlic','onion','beans'],
['garlic','onion','milk'],
['garlic','onion'],
['garlic','chili'],
['garlic','chili'],
['milk','onion','egg'],
['milk','onion','egg'],
['milk','onion','egg','bread'],
['milk','onion','chili','bread'],
]

# Use Transaction encoder to transform into an 1-hot boolean encoded numpy array
te = TransactionEncoder()
te_ary = te.fit(theData).transform(theData)

print("te_aray returned from Transactionencoder.fit().transform()")
print(te_ary)

te_aray returned from Transactionencoder.fit().transform()
[[False  True  True  True False  True  True]
 [False  True  True  True False  True False]
 [ True False  True False  True  True False]
 [ True False  True  True False  True False]
 [False  True False  True False  True  True]
 [False False False  True False  True  True]
 [ True False False False  True False  True]
 [ True False False False  True False  True]
 [ True False False False  True False  True]
 [ True False False False  True False  True]
 [ True False  True False  True False  True]
 [ True False  True False  True False  True]
 [False False False False  True  True  True]
 [False False False False  True False  True]
 [False False  True False  True False False]
 [False False  True False  True False False]
 [False False False  True False  True  True]
 [False False False  True False  True  True]
 [False  True False  True False  True  True]
 [False  True  True False False  True  True]]


In [58]:
# Convert into a dataframe for convenience and to pas into the apriori
df2 = pd.DataFrame(te_ary, columns=te.columns_)

print("DataFrame Version:")
print(df2.head(25))

DataFrame Version:
    beans  bread  chili    egg  garlic   milk  onion
0   False   True   True   True   False   True   True
1   False   True   True   True   False   True  False
2    True  False   True  False    True   True  False
3    True  False   True   True   False   True  False
4   False   True  False   True   False   True   True
5   False  False  False   True   False   True   True
6    True  False  False  False    True  False   True
7    True  False  False  False    True  False   True
8    True  False  False  False    True  False   True
9    True  False  False  False    True  False   True
10   True  False   True  False    True  False   True
11   True  False   True  False    True  False   True
12  False  False  False  False    True   True   True
13  False  False  False  False    True  False   True
14  False  False   True  False    True  False  False
15  False  False   True  False    True  False  False
16  False  False  False   True   False   True   True
17  False  False  False   T

In [81]:
# Call apriori to find frequent itemsets with min_support = 30%
freq_items = apriori(df2, min_support=0.3, use_colnames=True)
freq_items['length'] = freq_items['itemsets'].apply(lambda x: len(x))
freq_items.sort_values(['support', 'length'], inplace=True, ascending=False)
print("Frequent Items:")
print(freq_items)

Frequent Items:
    support                itemsets  length
5      0.75                 (onion)       1
3      0.55                (garlic)       1
4      0.55                  (milk)       1
1      0.45                 (chili)       1
8      0.40             (milk, egg)       2
10     0.40         (garlic, onion)       2
11     0.40           (milk, onion)       2
0      0.40                 (beans)       1
2      0.40                   (egg)       1
6      0.35         (beans, garlic)       2
12     0.30  (beans, garlic, onion)       3
13     0.30      (milk, egg, onion)       3
7      0.30          (beans, onion)       2
9      0.30            (egg, onion)       2


In [60]:
# Examples of How to filter your itemsets further, for illustration only, not actually used below
reducedFreqItems = freq_items[(((freq_items['length'] == 2) & (freq_items['support'] >= 0.40)) |
                               ((freq_items['length'] >= 3) & (freq_items['support'] >= 0.30 )))]

print("\n\nReduced freq_items (length == 2 & support >= 40%) | (length >=3 & support >= 30%) ")
print(reducedFreqItems)



Reduced freq_items (length == 2 & support >= 40%) | (length >=3 & support >= 30%) 
    support                itemsets  length
8       0.4             (milk, egg)       2
10      0.4         (garlic, onion)       2
11      0.4           (milk, onion)       2
12      0.3  (beans, garlic, onion)       3
13      0.3      (milk, egg, onion)       3


In [61]:
# Mining the rules by calling association_rules
print("The Rules")
rules = association_rules(freq_items, metric="confidence", min_threshold=0.7)
rules.columns = ['ante', 'consequ', 'antsup', 'consup', 'sup', 'conf', 'lift', 'leverage', 'conviction']
print(rules[["ante","consequ","antsup","consup","sup","conf","lift"]])

The Rules
               ante          consequ  antsup  consup   sup      conf      lift
0           (beans)         (garlic)    0.40    0.55  0.35  0.875000  1.590909
1           (beans)          (onion)    0.40    0.75  0.30  0.750000  1.000000
2            (milk)            (egg)    0.55    0.40  0.40  0.727273  1.818182
3             (egg)           (milk)    0.40    0.55  0.40  1.000000  1.818182
4             (egg)          (onion)    0.40    0.75  0.30  0.750000  1.000000
5          (garlic)          (onion)    0.55    0.75  0.40  0.727273  0.969697
6            (milk)          (onion)    0.55    0.75  0.40  0.727273  0.969697
7   (beans, garlic)          (onion)    0.35    0.75  0.30  0.857143  1.142857
8    (beans, onion)         (garlic)    0.30    0.55  0.30  1.000000  1.818182
9   (garlic, onion)          (beans)    0.40    0.40  0.30  0.750000  1.875000
10          (beans)  (garlic, onion)    0.40    0.40  0.30  0.750000  1.875000
11      (milk, egg)          (onion)    0.

Questions 1: <br>
Using the values already set in the code  <br>
(i.e. apriori min_support = 0.3, association_rule min_threshold = 0.7) <br>
look at the result an answer: <br>

a). Which 1-itemset has the greatest support?

In [92]:
freq_items.sort_values(['support', 'length'], inplace=True, ascending=False)
print(freq_items[((freq_items['length'] == 1) & (freq_items['support'] == freq_items.support.max()))])

   support itemsets  length
5     0.75  (onion)       1
